In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as np
from datetime import datetime
%matplotlib inline

sales_train = pd.read_csv('sales_train.csv')
sales_train.iloc[:, 0] = pd.to_datetime(sales_train.iloc[:, 0], format="%d.%m.%Y")

holidays = pd.read_feather('hol.feather')
item_en = pd.read_feather('it_en.feather')
shops_en = pd.read_feather('sh_en.feather')
item_categories_en = pd.read_feather('itc_en.feather')
holidays = holidays.rename(columns={'h_name': 'holiday_name', 'h_type': 'holiday_type'})
holidays['Is_Holiday'] = pd.Series('Yes', index=holidays.index)
item_en = item_en.rename(columns={'it_name': 'item_name', 'it_id': 'item_id', 'itc_id': 'item_category_id', 
                        'en_it_name': 'item_name_en'})
shops_en = shops_en.rename(columns={'sname': 'shop_name', 'sid': 'shop_id', 'en_sname': 'shop_name_en'})
item_categories_en = item_categories_en.rename(columns={'itc_name': 'item_category_name', 'itc_id': 'item_category_id',
                                                        'en_itc_name': 'item_category'})

train = pd.merge(sales_train, item_en, on='item_id', how='left')
train = pd.merge(train, shops_en, on='shop_id', how='left')
train = pd.merge(train, item_categories_en, on='item_category_id', how='left')
train = pd.merge(train, holidays, on='date', how='left')

# train['year'] = pd.DatetimeIndex(train['date']).year
# train['month'] = pd.DatetimeIndex(train['date']).month
# train['day'] = pd.DatetimeIndex(train['date']).day

train.head(2)

In [22]:
#Advice from week#2, group monthly and predict sales of the month from previous month benchmark
train_simple = train[['date_block_num', 'shop_id', 'item_id', 'item_cnt_day']]
train_year_month = train_simple.groupby(['date_block_num', 'shop_id', 'item_id'])['item_cnt_day'].sum()
train_year_month.sample(5)

year  month  shop_id  item_id
2013  5      25       9921       5.0
2014  11     57       18587      1.0
      4      19       4565       1.0
      10     45       786        2.0
2013  2      31       8633       1.0
Name: item_cnt_day, dtype: float64

In [ ]:
shops_en

In [14]:
#Add transaction rows that only shown in test to train.
test = pd.read_csv('test.csv')
test['shop_item'] = test[['shop_id', 'item_id']].astype(str).apply(lambda x: '_'.join(x), axis=1)
train['shop_item'] = train[['shop_id', 'item_id']].astype(str).apply(lambda x: '_'.join(x), axis=1)
test_extra = pd.DataFrame(data={'shop_item': test.shop_item})
train = pd.merge(train, test_extra, on = 'shop_item', how='outer')
train['shop_id'] = train['shop_item'].apply(lambda x: x.split('_')[0]).astype(int)
train['item_id'] = train['shop_item'].apply(lambda x: x.split('_')[1]).astype(int)
train.head(5)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_name,item_category_id,item_name_en,shop_name,shop_name_en,item_category_name,item_category,holiday_name,holiday_type,Is_Holiday,year,month,day,shop_item
0,2013-01-02,0.0,59,22154,999.00,1.0,ЯВЛЕНИЕ 2012 BD,37,SCENE 2012 BD,Ярославль ТЦ Альтаир,Yaroslavl Shopping Center Altair,Кино Blu Ray,Blu Ray Movie,New Year Holiday Week,National holiday,Yes,2013.0,1.0,2.0,59_22154
1,2013-01-03,0.0,25,2552,899.00,1.0,DEEP PURPLE The House Of Blue Light LP,58,DEEP PURPLE The House Of Blue Light LP,Москва ТРК Атриум,Moscow TRK Atrium,Музыка Винил,Music Vinyl,New Year Holiday Week,National holiday,Yes,2013.0,1.0,3.0,25_2552
2,2013-01-05,0.0,25,2552,899.00,-1.0,DEEP PURPLE The House Of Blue Light LP,58,DEEP PURPLE The House Of Blue Light LP,Москва ТРК Атриум,Moscow TRK Atrium,Музыка Винил,Music Vinyl,NaN,NaN,NaN,2013.0,1.0,5.0,25_2552
3,2013-01-06,0.0,25,2554,1709.05,1.0,DEEP PURPLE Who Do You Think We Are LP,58,DEEP PURPLE Who Do You Think We Are LP,Москва ТРК Атриум,Moscow TRK Atrium,Музыка Винил,Music Vinyl,NaN,NaN,NaN,2013.0,1.0,6.0,25_2554
4,2013-01-15,0.0,25,2555,1099.00,1.0,DEEP PURPLE 30 Very Best Of 2CD Фирм,56,DEEP PURPLE 30 Very Best Of 2CD Firms,Москва ТРК Атриум,Moscow TRK Atrium,Музыка CD фирменного производства,Music CD brand production,NaN,NaN,NaN,2013.0,1.0,15.0,25_2555


In [8]:
train.shape
train.isnull().sum()

date                   102796
date_block_num         102796
shop_id                     0
item_id                     0
item_price             102796
item_cnt_day           102796
item_name              102796
item_category_id       102796
item_name_en           102796
shop_name              102796
shop_name_en           102796
item_category_name     102796
item_category          102796
holiday_name          2714940
holiday_type          2714940
Is_Holiday            2714940
year                   102796
month                  102796
day                    102796
shop_item                   0
dtype: int64

In [36]:
#fill rows with nan
#fill item_price with mean
train['item_cnt_day'] = train['item_cnt_day'].fillna(0)
train['item_price'] = train.groupby('item_id')['item_price'].apply(lambda x: x.fillna(x.mean())) #85% filled
train['item_price'] = train['item_price'].fillna(0)
train['item_price'].isnull().sum()

0

In [37]:
train[train['item_id']==20599].sample(5)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_name,item_category_id,item_name_en,shop_name,shop_name_en,item_category_name,item_category,holiday_name,holiday_type,Is_Holiday,year,month,day,shop_item
2973057,NaT,NaN,15,20599,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15_20599
2977668,NaT,NaN,18,20599,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18_20599
2984225,NaT,NaN,42,20599,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42_20599
2998458,NaT,NaN,48,20599,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48_20599
3005139,NaT,NaN,59,20599,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59_20599


In [24]:
train[train['item_price'].isnull()].sample(5)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_name,item_category_id,item_name_en,shop_name,shop_name_en,item_category_name,item_category,holiday_name,holiday_type,Is_Holiday,year,month,day,shop_item
3034965,NaT,NaN,44,20599,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44_20599
2984094,NaT,NaN,42,8676,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42_8676
2994416,NaT,NaN,52,14760,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52_14760
2990921,NaT,NaN,53,12802,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53_12802
2961471,NaT,NaN,31,16976,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31_16976


In [ ]:
#features to be added: Latitude, Longitude,
#training data to be added: add transaction 0 for shop_id and item_id combinations not included in training but included in test


train['']

In [79]:
item_en.head()

,it_name,it_id,itc_id,en_it_name
0,ВО ВЛАСТИ НАВАЖДЕНИЯ ПЛАСТ D,0.0,40.0,IN THE FIRE OF DREATING THE PLAST D
1,ABBYY FineReader 12 Professional Edition Full ...,1.0,76.0,ABBYY FineReader 12 Professional Edition Full ...
2,В ЛУЧАХ СЛАВЫ UNV D,2.0,40.0,In the glories of the glory UNV D
3,ГОЛУБАЯ ВОЛНА Univ D,3.0,40.0,BLUE WAVE Univ D
4,КОРОБКА СТЕКЛО D,4.0,40.0,BOX GLASS D


In [ ]:
import multiprocessing as mp
print (mp.cpu_count())